In [2]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer

data1 = "I'm designing a document and don't want to get bogged down in what the text actually says"
data2 = "I'm creating a template for various paragraph styles and need to see what they will look like."
data3 = "I'm trying to learn more about some features of Microsoft Word and don't want to practice on a real document"

df1 = pd.DataFrame({'First_Para':[data1], 'Second_Para':[data2], 'Third_Para':[data3]})
vectorizer = TfidfVectorizer()
doc_vec = vectorizer.fit_transform(df1.iloc[0])

df2 = pd.DataFrame(doc_vec.toarray().transpose(),index=vectorizer.get_feature_names_out())

df2.columns = df1.columns
print(df2)

           First_Para  Second_Para  Third_Para
about        0.000000     0.000000    0.254170
actually     0.288540     0.000000    0.000000
and          0.170416     0.162095    0.150117
bogged       0.288540     0.000000    0.000000
creating     0.000000     0.274451    0.000000
designing    0.288540     0.000000    0.000000
document     0.219442     0.000000    0.193303
don          0.219442     0.000000    0.193303
down         0.288540     0.000000    0.000000
features     0.000000     0.000000    0.254170
for          0.000000     0.274451    0.000000
get          0.288540     0.000000    0.000000
in           0.288540     0.000000    0.000000
learn        0.000000     0.000000    0.254170
like         0.000000     0.274451    0.000000
look         0.000000     0.274451    0.000000
microsoft    0.000000     0.000000    0.254170
more         0.000000     0.000000    0.254170
need         0.000000     0.274451    0.000000
of           0.000000     0.000000    0.254170
on           

## BM25

In [3]:
from rank_bm25 import BM25Okapi

corpus=[
    "I will take the ring, though I do not know the way.",
    "I will help you bear this burden, Frodo Baggins, as long as it is yours to bear",
    "If by my life or death I can protect you, I will."
]

tokenized_corpus = [doc.split(" ") for doc in corpus]

bm25 = BM25Okapi(tokenized_corpus)

In [4]:
query="I will take"
tokenized_query = query.split(" ")
doc_scores = bm25.get_scores(tokenized_query)
print(doc_scores)

[0.79842666 0.18266174 0.15070007]


## LDA

In [5]:
# Importing modules
import pandas as pd
# Read Corona Tweets
tweets = pd.read_csv(r"Corona_NLP_train.csv", encoding='latin-1')
# Print head
tweets.head()

,ï»¿UserName,ScreenName,Location,TweetAt,OriginalTweet,Sentiment
0,3799,48751,London,16-03-2020,@MeNyrbie @Phil_Gahan @Chrisitv https://t.co/i...,Neutral
1,3800,48752,UK,16-03-2020,advice Talk to your neighbours family to excha...,Positive
2,3801,48753,Vagabonds,16-03-2020,Coronavirus Australia: Woolworths to give elde...,Positive
3,3802,48754,NaN,16-03-2020,My food stock is not the only one which is emp...,Positive
4,3803,48755,NaN,16-03-2020,"Me, ready to go at supermarket during the #COV...",Extremely Negative


In [6]:
# Remove the columns
tweets = tweets.drop(columns=['ï»¿UserName', 'ScreenName', 'Location','TweetAt','Sentiment'], axis=1).sample(100)
# Print out the first rows of papers
tweets.head()

,OriginalTweet
40558,In the COVID induced scramble for toilet paper...
11342,A lady in a supermarket wearing a surgical mas...
5990,I wonder if the hoarders are renting storage u...
18561,Observations from a visit to the grocery store...
9905,"#easyjet and #jet2 share prices up, jet2 up 66..."


In [7]:
# Load the regular expression library
import re
# Remove punctuation
tweets['OriginalTweet_processed'] = \
tweets['OriginalTweet'].map(lambda x: re.sub('[@#,\\.!?]', '', x))
# Convert the tweets to lowercase
tweets['OriginalTweet_processed'] = \
tweets['OriginalTweet_processed'].map(lambda x: x.lower())
# Print out the first rows of tweets
tweets['OriginalTweet_processed'].head()

40558    in the covid induced scramble for toilet paper...
11342    a lady in a supermarket wearing a surgical mas...
5990     i wonder if the hoarders are renting storage u...
18561    observations from a visit to the grocery store...
9905     easyjet and jet2 share prices up jet2 up 66% w...
Name: OriginalTweet_processed, dtype: object

In [8]:
import gensim
from gensim.utils import simple_preprocess
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
stop_words.extend(['https', 'tco'])
def sent_to_words(sentences):
    for sentence in sentences:
        # deacc=True removes punctuations
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) 
             if word not in stop_words] for doc in texts]
data = tweets.OriginalTweet_processed.values.tolist()
data_words = list(sent_to_words(data))
# remove stop words
data_words = remove_stopwords(data_words)
print(data_words[:1][0][:30])

['covid', 'induced', 'scramble', 'toilet', 'paper', 'amp', 'groceries', 'heard', 'plenty', 'hardworking', 'supermarket', 'staff', 'truck', 'drivers', 'stepped', 'says', 'truckie', 'riley', 'topping']


[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/pineapple/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [9]:
import gensim.corpora as corpora
# Create Dictionary
id2word = corpora.Dictionary(data_words)
# Create Corpus
texts = data_words
# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]
# View
print(corpus[:1][0][:30])

[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1), (6, 1), (7, 1), (8, 1), (9, 1), (10, 1), (11, 1), (12, 1), (13, 1), (14, 1), (15, 1), (16, 1), (17, 1), (18, 1)]


In [10]:
from pprint import pprint
# number of topics
num_topics = 7
# Build LDA model
lda_model = gensim.models.LdaMulticore(corpus=corpus,
                                       id2word=id2word,
                                       num_topics=num_topics)
# Print the Keyword in the 10 topics
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

[(0,
  '0.009*"coronavirus" + 0.007*"reports" + 0.006*"right" + 0.006*"people" + '
  '0.006*"prices" + 0.005*"dog" + 0.005*"looking" + 0.005*"amp" + '
  '0.004*"really" + 0.004*"us"'),
 (1,
  '0.016*"coronavirus" + 0.012*"covid" + 0.009*"grocery" + 0.008*"would" + '
  '0.008*"store" + 0.008*"idea" + 0.007*"supermarket" + 0.007*"panic" + '
  '0.007*"food" + 0.006*"measures"'),
 (2,
  '0.021*"coronavirus" + 0.013*"covid" + 0.011*"food" + 0.008*"pandemic" + '
  '0.008*"consumer" + 0.007*"shortages" + 0.007*"prices" + 0.006*"paper" + '
  '0.006*"toilet" + 0.006*"survive"'),
 (3,
  '0.026*"coronavirus" + 0.019*"covid" + 0.018*"supermarket" + 0.012*"food" + '
  '0.009*"covid_" + 0.007*"outbreak" + 0.006*"stock" + 0.006*"shopping" + '
  '0.006*"staff" + 0.005*"nhs"'),
 (4,
  '0.023*"coronavirus" + 0.018*"covid" + 0.010*"china" + 0.009*"industry" + '
  '0.009*"supermarket" + 0.008*"pandemic" + 0.008*"prices" + 0.008*"store" + '
  '0.006*"get" + 0.006*"retail"'),
 (5,
  '0.013*"food" + 0.013*"c

In [11]:
import pyLDAvis.gensim
import pickle 
import pyLDAvis
import os
# Visualize the topics
pyLDAvis.enable_notebook()
LDAvis_data_filepath = os.path.join('./results/ldavis_prepared_'+str(num_topics))
LDAvis_prepared = pyLDAvis.gensim.prepare(lda_model, corpus, id2word)

LDAvis_prepared

/Users/pineapple/opt/anaconda3/envs/NLP/lib/python3.12/site-packages/joblib/externals/loky/backend/fork_exec.py:38: DeprecationWarning: This process (pid=71702) is multi-threaded, use of fork() may lead to deadlocks in the child.
  pid = os.fork()
/Users/pineapple/opt/anaconda3/envs/NLP/lib/python3.12/site-packages/joblib/externals/loky/backend/fork_exec.py:38: DeprecationWarning: This process (pid=71702) is multi-threaded, use of fork() may lead to deadlocks in the child.
  pid = os.fork()
/Users/pineapple/opt/anaconda3/envs/NLP/lib/python3.12/site-packages/joblib/externals/loky/backend/fork_exec.py:38: DeprecationWarning: This process (pid=71702) is multi-threaded, use of fork() may lead to deadlocks in the child.
  pid = os.fork()
/Users/pineapple/opt/anaconda3/envs/NLP/lib/python3.12/site-packages/joblib/externals/loky/backend/fork_exec.py:38: DeprecationWarning: This process (pid=71702) is multi-threaded, use of fork() may lead to deadlocks in the child.
  pid = os.fork()
/Users/p

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
4     -0.141524  0.029040       1        1  20.067562
2      0.058186  0.051585       2        1  17.789255
6     -0.010317 -0.130153       3        1  15.423663
3      0.007768  0.064273       4        1  14.275943
5      0.080075 -0.008967       5        1  13.824655
1      0.002978  0.005330       6        1  11.555026
0      0.002835 -0.011107       7        1   7.063897, topic_info=            Term       Freq      Total Category  logprob  loglift
66          food  13.000000  13.000000  Default  30.0000  30.0000
238        would   4.000000   4.000000  Default  29.0000  29.0000
308       covid_   6.000000   6.000000  Default  28.0000  28.0000
56      industry   3.000000   3.000000  Default  27.0000  27.0000
271          ita   3.000000   3.000000  Default  26.0000  26.0000
..           ...        ...        ...      ...      ...      ...
21   coronavirus   1.209897  33.767625   Topic7  -4.6664  -0.6788
59        prices   0.712629  10.629244   Topic7  -5.1958  -0.0522
0            amp   0.588117   9.111207   Topic7  -5.3878  -0.0902
98        really   0.552470   3.119405   Topic7  -5.4503   0.9192
285        today   0.552341   6.055401   Topic7  -5.4506   0.2556

[412 rows x 6 columns], token_table=      Topic      Freq            Term
term                                 
989       2  0.513313         account
543       2  0.778590      acgrayling
702       5  0.859023          across
1033      1  0.781780          advice
1015      4  0.832939        affected
...     ...       ...             ...
238       6  0.468027           would
756       3  0.813052           wwork
288       7  0.920151  yellowstonenps
941       3  0.813052              zd
542       1  0.511518            zero

[487 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[5, 3, 7, 4, 6, 2, 1])

## LSA

In [12]:
import os.path
from gensim import corpora
from gensim.models import LsiModel
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from gensim.models.coherencemodel import CoherenceModel
import matplotlib.pyplot as plt

In [13]:
import gensim
from gensim.utils import simple_preprocess
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
stop_words.extend(['https', 'tco'])
def sent_to_words(sentences):
    for sentence in sentences:
        # deacc=True removes punctuations
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) 
             if word not in stop_words] for doc in texts]
def stem_words(texts):
    return [[p_stemmer.stem(word) for word in simple_preprocess(str(doc))
             ] for doc in texts]

data = tweets.OriginalTweet_processed.values.tolist()
data_words = list(sent_to_words(data))
# remove stop words
data_words = remove_stopwords(data_words)
print(data_words[:1][0][:30])

['covid', 'induced', 'scramble', 'toilet', 'paper', 'amp', 'groceries', 'heard', 'plenty', 'hardworking', 'supermarket', 'staff', 'truck', 'drivers', 'stepped', 'says', 'truckie', 'riley', 'topping']


[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/pineapple/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [14]:
doc_term_matrix = [id2word.doc2bow(twt) for twt in data_words]

In [15]:
lsa_model = LsiModel(doc_term_matrix, num_topics=num_topics, id2word = id2word)
print(lsa_model.print_topics(num_topics=num_topics, num_words=10))

/Users/pineapple/opt/anaconda3/envs/NLP/lib/python3.12/site-packages/gensim/models/lsimodel.py:963: DeprecationWarning: `scipy.sparse.sparsetools.csc_matvecs` is deprecated along with the `scipy.sparse.sparsetools` namespace. `scipy.sparse.sparsetools.csc_matvecs` will be removed in SciPy 1.14.0, and the `scipy.sparse.sparsetools` namespace will be removed in SciPy 2.0.0.
  sparsetools.csc_matvecs(


[(0, '0.598*"coronavirus" + 0.442*"covid" + 0.186*"food" + 0.177*"supermarket" + 0.160*"pandemic" + 0.155*"amp" + 0.149*"prices" + 0.120*"grocery" + 0.119*"consumer" + 0.101*"store"'), (1, '-0.527*"coronavirus" + 0.340*"covid" + 0.240*"food" + 0.165*"amp" + 0.156*"help" + 0.149*"workers" + 0.147*"supermarket" + 0.135*"like" + -0.129*"prices" + 0.121*"people"'), (2, '-0.470*"covid" + 0.437*"food" + 0.219*"amp" + 0.153*"coronavirus" + -0.151*"pandemic" + 0.110*"people" + -0.102*"prices" + 0.102*"demand" + 0.099*"empty" + 0.095*"day"'), (3, '-0.405*"china" + -0.379*"amp" + 0.261*"supermarket" + -0.241*"retail" + -0.236*"apple" + -0.157*"locations" + -0.140*"mckinsey" + -0.128*"store" + 0.105*"empty" + 0.105*"shelves"'), (4, '0.358*"store" + 0.341*"grocery" + -0.266*"prices" + 0.243*"like" + 0.149*"supermarket" + -0.146*"amp" + 0.142*"get" + -0.139*"pandemic" + -0.132*"food" + 0.127*"apple"'), (5, '-0.279*"prices" + 0.263*"supermarket" + -0.255*"grocery" + -0.217*"store" + 0.187*"workers" 

## NMF

In [16]:
import pandas as pd
from sklearn.decomposition import NMF
from sklearn.feature_extraction.text import TfidfVectorizer
 
tweets = pd.read_csv(r"Corona_NLP_train.csv", encoding='latin-1')
# Print head
tweets.head()

,ï»¿UserName,ScreenName,Location,TweetAt,OriginalTweet,Sentiment
0,3799,48751,London,16-03-2020,@MeNyrbie @Phil_Gahan @Chrisitv https://t.co/i...,Neutral
1,3800,48752,UK,16-03-2020,advice Talk to your neighbours family to excha...,Positive
2,3801,48753,Vagabonds,16-03-2020,Coronavirus Australia: Woolworths to give elde...,Positive
3,3802,48754,NaN,16-03-2020,My food stock is not the only one which is emp...,Positive
4,3803,48755,NaN,16-03-2020,"Me, ready to go at supermarket during the #COV...",Extremely Negative


In [17]:
# Remove the columns
tweets = tweets.drop(columns=['ï»¿UserName', 'ScreenName', 'Location','TweetAt','Sentiment'], axis=1).sample(100)
# Print out the first rows of papers
tweets.head()

,OriginalTweet
7417,"In the spirit of caution I share:\r\r\n""I just..."
11717,Social distancing!!!! Apparently not when you ...
506,NEW: Coronavirus package hits roadblocks amid ...
15022,Banks of all sizes across the U S have already...
13354,If you are currently working at a grocery stor...


In [18]:
# use tfidf by removing tokens that don't appear in at least 50 documents
vect = TfidfVectorizer(min_df=10, stop_words=stop_words )
 
# Fit and transform
X = vect.fit_transform(tweets.OriginalTweet)

In [19]:
# Create an NMF instance: model
# the 10 components will be the topics
model = NMF(n_components=10, random_state=5)
 
# Fit the model to TF-IDF
model.fit(X)
 
# Transform the TF-IDF: nmf_features
nmf_features = model.transform(X)

In [20]:
nmf_features

array([[0.00000000e+00, 7.25644272e-02, 7.80726740e-02, 0.00000000e+00,
        0.00000000e+00, 0.00000000e+00, 2.36022117e-01, 0.00000000e+00,
        0.00000000e+00, 0.00000000e+00],
       [0.00000000e+00, 0.00000000e+00, 4.09545425e-02, 0.00000000e+00,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        5.53463015e-01, 0.00000000e+00],
       [1.11645430e-01, 1.08337128e-01, 0.00000000e+00, 0.00000000e+00,
        3.67946979e-05, 0.00000000e+00, 6.32280045e-05, 0.00000000e+00,
        4.66108970e-05, 0.00000000e+00],
       [9.93648505e-05, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00, 2.07579570e-04, 0.00000000e+00, 4.32348030e-01,
        0.00000000e+00, 0.00000000e+00],
       [8.77488135e-06, 0.00000000e+00, 7.48083039e-02, 1.89015753e-02,
        5.61891427e-02, 0.00000000e+00, 5.95388285e-02, 1.72639552e-01,
        5.85652781e-02, 8.69397763e-02],
       [2.81106652e-02, 0.00000000e+00, 1.09132946e-02, 1.61921317e-01,
   

In [24]:
components_df = pd.DataFrame(model.components_, columns=vect.get_feature_names_out())
components_df

,19,co,consumer,coronavirus,covid,covid19,covid_19,food,get,grocery,prices,store,supermarket,work
0,0.000000,10.177027,0.002321,0.000000,0.093572,0.000000,0.525682,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,2.507881,0.010706,0.000000,0.000000,2.427836,0.000000,0.000000,0.042228,0.000000,0.000000,0.001970,0.098949,0.000000,0.112721
2,0.049870,0.000000,0.000000,0.000000,0.000000,0.000000,0.020648,0.000000,0.023174,1.835004,0.000000,1.973962,0.000000,0.000000
3,0.081841,0.000000,0.000000,0.000563,0.000000,0.000000,0.127647,0.000000,0.112686,0.000000,0.000000,0.000000,2.961793,0.000000
4,0.000000,0.000000,0.000000,0.002429,0.003636,0.000000,0.206960,0.000000,0.120772,0.000000,2.809617,0.000000,0.000000,0.000000
5,0.000000,0.007911,0.000000,3.295499,0.000000,0.000000,0.000000,0.021892,0.000000,0.110849,0.000000,0.000000,0.000000,0.000000
6,0.113752,0.001957,3.346553,0.000785,0.000000,0.001444,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
7,0.000000,0.000000,0.000000,0.000000,0.001481,0.000000,0.853044,1.845793,0.040871,0.000000,0.000000,0.000000,0.000000,0.000000
8,0.000000,0.000000,0.000000,0.000000,0.006016,0.000000,0.000000,0.000000,0.916713,0.000000,0.000000,0.013412,0.000000,1.158340
9,0.000000,0.018383,0.000181,0.000000,0.000000,1.543470,0.000000,0.038964,0.055644,0.064211,0.000796,0.000000,0.000000,0.000000


In [25]:
for topic in range(components_df.shape[0]):
    tmp = components_df.iloc[topic]
    print(f'For topic {topic+1} the words with the highest value are:')
    print(tmp.nlargest(10))
    print('\n')

For topic 1 the words with the highest value are:
co             10.177027
covid_19        0.525682
covid           0.093572
consumer        0.002321
19              0.000000
coronavirus     0.000000
covid19         0.000000
food            0.000000
get             0.000000
grocery         0.000000
Name: 0, dtype: float64


For topic 2 the words with the highest value are:
19             2.507881
covid          2.427836
work           0.112721
store          0.098949
food           0.042228
co             0.010706
prices         0.001970
consumer       0.000000
coronavirus    0.000000
covid19        0.000000
Name: 1, dtype: float64


For topic 3 the words with the highest value are:
store          1.973962
grocery        1.835004
19             0.049870
get            0.023174
covid_19       0.020648
co             0.000000
consumer       0.000000
coronavirus    0.000000
covid          0.000000
covid19        0.000000
Name: 2, dtype: float64


For topic 4 the words with the highest val

## PCA

In [26]:
from nltk.corpus import stopwords
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
stop_words.extend(['https','tco'])

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/pineapple/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [27]:
vect= TfidfVectorizer(stop_words=stop_words)
x = vect.fit_transform(tweets.OriginalTweet)
tf_idf_vect = pd.DataFrame(x.toarray().transpose(),index=vect.get_feature_names_out())


In [28]:
tf_idf_vect

,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0
100,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0
11,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0
13,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.261114,0.0,0.0,0.0,0.0
16nh5wuzox,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.224157,0.000000,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
youâ,0.0,0.246301,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0
zero,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0
ziieacwfog,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0
zolnefkhxz,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0


In [29]:
from sklearn.decomposition import PCA
pca = PCA(n_components=50)
pca.fit_transform(tf_idf_vect)

array([[-8.74238154e-03,  8.17141958e-03, -5.20440595e-03, ...,
        -1.16529795e-02,  3.36458466e-03,  3.43007099e-03],
       [-2.65201210e-02, -1.18381132e-02,  1.98217256e-02, ...,
        -6.99519554e-03, -1.33950788e-02, -1.65854994e-02],
       [-1.19882772e-02,  1.04809707e-02,  6.06281207e-03, ...,
         1.93271228e-02, -2.73434053e-02, -4.22264800e-02],
       ...,
       [ 2.71474338e-03,  2.12703094e-02, -6.44129440e-02, ...,
        -7.25817601e-02, -7.63194029e-02, -5.50823439e-02],
       [-3.31332653e-03,  1.46155612e-02, -4.92467412e-02, ...,
        -7.10570396e-03, -5.27280246e-03, -5.96378975e-03],
       [ 4.44965480e-02, -4.30826356e-02,  1.03095153e-02, ...,
        -1.66989088e-02,  3.82950078e-02, -8.87909650e-05]])

In [30]:
print(pca.components_)

[[-0.03703872  0.01700879  0.00674683 ... -0.01100549 -0.06096137
   0.04743106]
 [ 0.00066016 -0.09439248 -0.06130642 ... -0.01654728 -0.02264427
   0.03357699]
 [-0.0308215   0.09210414  0.06214952 ... -0.04229609  0.01084377
  -0.1572611 ]
 ...
 [ 0.13817685  0.06807946 -0.0016724  ... -0.01163301 -0.0072622
   0.02903595]
 [ 0.04503803  0.03013976  0.00981831 ...  0.06604964  0.10178815
  -0.02477647]
 [ 0.09573981  0.09297686 -0.22141623 ... -0.01881744  0.07742084
  -0.00486171]]


## SVD

In [31]:
import numpy as np

In [32]:
vect= TfidfVectorizer(stop_words=stop_words, smooth_idf=True)
x = vect.fit_transform(tweets.OriginalTweet).todense()
x = np.asarray(x)

In [33]:
from sklearn.decomposition import TruncatedSVD
svd_modeling = TruncatedSVD(n_components=4, algorithm='randomized', n_iter=100, random_state=122)
svd_modeling.fit(x)
components=svd_modeling.components_
vocab = vect.get_feature_names_out()

In [34]:
topic_word_list = []
def get_topics(components):
    for i, comp in enumerate(components):
        terms_comp =zip(vocab,comp)
        sorted_terms = sorted(terms_comp, key= lambda x:x[1], reverse=True)[:7]
        topic=" "
        for t in sorted_terms:
            topic= topic + ' ' + t[0]
        topic_word_list.append(topic)
        print(topic_word_list)
    return topic_word_list
get_topics(components)
        

['  co coronavirus store 19 grocery covid food']
['  co coronavirus store 19 grocery covid food', '  store grocery food time panic supermarkets go']
['  co coronavirus store 19 grocery covid food', '  store grocery food time panic supermarkets go', '  19 covid store grocery consumer behavior shifts']
['  co coronavirus store 19 grocery covid food', '  store grocery food time panic supermarkets go', '  19 covid store grocery consumer behavior shifts', '  co coronavirus grocery stores store prices bueqsjzc5l']


['  co coronavirus store 19 grocery covid food',
 '  store grocery food time panic supermarkets go',
 '  19 covid store grocery consumer behavior shifts',
 '  co coronavirus grocery stores store prices bueqsjzc5l']